# Data Quality Assurance Project

### Author: William Olichney, Research Data Associate, AidData

While preforming quality assurance on a coder's countries, I noticed that they often captured the commitment amount of loans. They interpreted the commitment amounts to be amount outstanding. This error spanned at least three DDC sheets. Since the coder was tasked with large countries, such as Indonesia, hand checking the amount outstanding values is not feasible. 

So, this workflow attempts to highlight suspicious values by leveraging AidData's internal database, which will serve as the benchmark to see if the observed data entry was amount outstanding, or the commitment amount. The benefit of this code over hand checking is that it automatically handles tasks like historical exchange rates, if the coder did not pick up that the loan was cofinanced or not, and if data observation matches the commitment amount.

However, at this current stage, someone would need to hand check these suspicious amounts. So, this workflow will attempt to automate AI prompting to create an additional layer of validation.

In [1]:
# Install Libraries
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime

## Importing Data

At this stage of the workflow, I would import data one country at a time for a check. This will make it easier as the DDC spreaedsheets are already seperated by country. This could be better automated one day, and I think Ameya Joshi has a print out of the full data with all the countries together. One could simply plug in that dataframe.

### Step 1:
Download QA country's DDC sheet as a csv file. Alter code under ##DDC to reflect the filepath of this downloaded csv file

### Step 2:
Download loan information from the internal and export as a csv file. Alter code under ##Internal to reflect the filepath of this downloaded csv file.

### Step 3:
Download QA country's DOCP sheet as a csv file, this will come into the workflow later.  Alter code under ##DOCP to reflect the filepath.

In [3]:
##DDC
qa_ddc_verbose = pd.read_csv("ddc.csv")

columns_to_drop = ["Parent DOCP ID", "DOCP ID", "Recipient Official Loan ID", "Exact Date?", "Observed Data Comment", "Under Review"]

# Drop only columns that exist
qa_ddc = qa_ddc_verbose.drop(columns=[col for col in columns_to_drop if col in qa_ddc_verbose.columns])

##Internal
internal_df_verbose = pd.read_csv('internal data pull.csv')

columns_keep = ['AidData Tuff Project ID', 'Commitment Year', 'Co-financed', 'Amount (Original Currency)', 'Original Currency',
               'Adjusted Amount (Nominal USD)', 'Commitment Date (MM/DD/YYYY)', 'Fixed Interest Rate', 'Amount (Nominal USD)']
internal_df = internal_df_verbose[columns_keep]
internal_df = internal_df.rename(columns={"AidData Tuff Project ID" : "AidData Record ID"})

##DOCP
docp_df = pd.read_csv('docp.csv')

#DOCP join ready
docp_df = docp_df.rename(columns={'AidData Source ID' : 'Source ID'})

# perform initial join
joined_data = pd.merge (qa_ddc, internal_df, on = 'AidData Record ID', how = "left")



## Data cleaning

In [5]:
##Remove commas from data
joined_data['Observed Amount'] = pd.to_numeric(joined_data['Observed Amount'].str.replace(',',''), errors = 'coerce')
joined_data[['Observed Amount', 'Amount (Nominal USD)']] = joined_data[['Observed Amount', 'Amount (Nominal USD)']].astype(float)
pd.set_option('display.float_format', '{:.2f}'.format)


## API for exchange rates
To make comparison easier between observed amount and amount outstanding, any observation that is not in USD will be converted to USD, so as to compare to the internal's column for Amount (Nominal USD)

#### To run this cell, you'll need an API key

**Go to:** https://openexchangerates.org/

In [57]:
api_key3 = 'enter your API key here'




api_load_in = {}

# Step 1: Populate `api_load_in` dictionary
for index, row in joined_data.iterrows():
    if row['Observed Amount Currency'] != 'USD':
        # Initialize list if the key does not exist
        if row['AidData Record ID'] not in api_load_in:
            api_load_in[row['AidData Record ID']] = []
        
        # Append the new observation to the list
        api_load_in[row['AidData Record ID']].append({
            'Date': row['Date'],
            'Observed Amount Currency': row['Observed Amount Currency']
        })

# Step 2: Process and format dates
unique_dates = set()
unique_currencies = set()

for aid_id, records in api_load_in.items():
    for record in records:
        # Use the date formatting function
        formatted_date = format_date(record['Date'])
        
        if formatted_date:
            record['Formatted Date'] = formatted_date  # Store formatted date back in the record
            unique_dates.add(formatted_date)
            unique_currencies.add(record['Observed Amount Currency'])
        else:
            print(f"Warning: Unrecognized date format for {record['Date']}")

# Step 3: Fetch exchange rates only once per date
exchange_rates = {}

for date in unique_dates:
    try:
        # Construct API URL
        url = f"https://openexchangerates.org/api/historical/{date}.json?app_id={api_key3}"
        print(f"Fetching exchange rates for {date}...")  # Debugging

        response = requests.get(url)

        # Check for API request failure
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            continue  # Skip to the next date

        data = response.json()

        # Store exchange rates for only relevant currencies
        if 'rates' in data:
            exchange_rates[date] = {currency: data['rates'].get(currency, None) for currency in unique_currencies}
        else:
            print(f"Warning: No exchange rates found for {date}")

        # Respect API rate limits
        time.sleep(1)

    except Exception as e:
        print(f"Error fetching data for {date}: {e}")

# Step 4: Update `api_load_in` with exchange rates
for aid_id, records in api_load_in.items():
    for record in records:
        formatted_date = record['Formatted Date']
        currency = record['Observed Amount Currency']

        # Get exchange rate for this date & currency
        exchange_rate = exchange_rates.get(formatted_date, {}).get(currency, None)
        
        # Update the record with the exchange rate
        record['Exchange Rate to USD'] = exchange_rate

# Step 5: Print updated `api_load_in`
print(api_load_in)

Fetching exchange rates for 2012-12-19...
Fetching exchange rates for 2016-12-31...
Fetching exchange rates for 2011-09-30...
Fetching exchange rates for 2013-12-31...
Fetching exchange rates for 2022-12-31...
Fetching exchange rates for 2015-12-31...
Fetching exchange rates for 2020-01-28...
Fetching exchange rates for 2010-08-13...
Fetching exchange rates for 2018-03-31...
Fetching exchange rates for 2015-11-06...
Fetching exchange rates for 2019-10-28...
Fetching exchange rates for 2022-01-21...
Fetching exchange rates for 2020-12-30...
Fetching exchange rates for 2015-10-30...
Fetching exchange rates for 2012-12-31...
Fetching exchange rates for 2019-06-30...
Fetching exchange rates for 2020-06-30...
Fetching exchange rates for 2014-12-31...
Fetching exchange rates for 2018-12-31...
Fetching exchange rates for 2010-12-31...
Fetching exchange rates for 2018-10-13...
Fetching exchange rates for 2021-11-23...
Fetching exchange rates for 2010-01-01...
Fetching exchange rates for 2022-0

## Joining exchange info to the data

In [70]:
exchange_panda_prep = []

for aid_id, records in api_load_in.items():
    for record in records:
        if isinstance(record, dict):  # Ensure record is a dictionary
            new_record = record.copy()  # Copy to avoid modifying original data
            new_record['AidData Record ID'] = aid_id  # Add the AidData Record ID
            exchange_panda_prep.append(new_record)

# Create DataFrame
exchange_df = pd.DataFrame(exchange_panda_prep)
exchange_df = exchange_df.drop(columns=('Formatted Date'))
exchange_df = exchange_df.drop_duplicates(
    subset=["Date", "Observed Amount Currency", "AidData Record ID"], keep="first"
)

exchange_df.to_csv("indonesia_exchange.csv",index=False) # So API does not have to be run everytime
exchange_df =pd.read_csv("indonesia_exchange.csv")

merged_data = joined_data.merge(
    exchange_df,
    how="left",
    left_on=["Date", "Observed Amount Currency", "AidData Record ID"],
    right_on=["Date", "Observed Amount Currency", "AidData Record ID"],
    suffixes=("", "_exchange")
)

# Step 2: Convert Observed Amount to USD if necessary
merged_data["Converted Amount"] = merged_data.apply(
    lambda row: row["Observed Amount"] / row["Exchange Rate to USD"]
    if row["Observed Amount Currency"] != "USD" else row["Observed Amount"],
    axis=1
)

### If you have already run the exchange information, use cell below

In [8]:
exchange_df =pd.read_csv("indonesia_exchange.csv")

merged_data = joined_data.merge(
    exchange_df,
    how="left",
    left_on=["Date", "Observed Amount Currency", "AidData Record ID"],
    right_on=["Date", "Observed Amount Currency", "AidData Record ID"],
    suffixes=("", "_exchange")
)

# Step 2: Convert Observed Amount to USD if necessary
merged_data["Converted Amount"] = merged_data.apply(
    lambda row: row["Observed Amount"] / row["Exchange Rate to USD"]
    if row["Observed Amount Currency"] != "USD" else row["Observed Amount"],
    axis=1
)

## As exhange rate information may not be perfect, set a tolerance, and see what matches commitment
Additionally, this step will include edge cases with problems relating to Co-financing

In [10]:
# Step 3: Define tolerance range (10% of the 'Amount (Nominal USD)')
tolerance = merged_data['Amount (Nominal USD)'] * 0.10

# Step 4: Identify valid matches
valid_matches = merged_data[
    ((merged_data["Converted Amount"] == merged_data["Amount (Nominal USD)"])) |

    # Standard match: Converted amount within ±3M range of nominal amount
    (merged_data["Converted Amount"].between(
        merged_data["Amount (Nominal USD)"] - tolerance,
        merged_data["Amount (Nominal USD)"] + tolerance
    )) |

    # Cofinanced case 1: Coder mistakenly entered full amount instead of 50%
    ((merged_data["Co-financed"] == "Yes") &
     merged_data["Converted Amount"].between(
         (merged_data["Amount (Nominal USD)"] * 0.5) - tolerance,
         (merged_data["Amount (Nominal USD)"] * 0.5) + tolerance
     )) |

    # Cofinanced case 2: Coder correctly divided observed amount by 2, but the database mistakenly kept full commitment amount
    ((merged_data["Co-financed"] == "Yes") &
     (merged_data["Converted Amount"]).between(
         (merged_data["Amount (Nominal USD)"]*2) - tolerance,
         (merged_data["Amount (Nominal USD)"]*2) + tolerance
     ))
]

final_columns = [
    "Source ID", "AidData Record ID", "Debt Data Type", "Date", "Observed Amount",
    "Observed Amount Currency", "Commitment Year", "Amount (Original Currency)",
    "Original Currency", "Amount (Nominal USD)", 'Converted Amount'
]
validated_data = valid_matches[final_columns]

## Now we have our suspicious data, let's add more information to give context to the loan

In [12]:
internal_df_info = internal_df_verbose.rename(columns={'AidData Tuff Project ID':'AidData Record ID'})
internal_and_ddc = validated_data.merge(internal_df_info, on = "AidData Record ID", how = 'left')
#join docp to internal and sus ddc
internal_docp_ddc = internal_and_ddc.merge(docp_df, on = 'Source ID', how = 'left')
cleaning_internal_docp_ddc = internal_docp_ddc.dropna(axis=1, how ='any')
# List of columns to drop
columns_to_drop = [
    'Source ID', 'Commitment Year_x', 'Amount (Original Currency)_x', 'Amount (Nominal USD)_x',
    'Recommended For Aggregates', 'Umbrella', 'Financier Country', 'Recipient ISO-3', 
    'Recipient Region', 'Commitment Year_y', 'Staff Comments', 'Status', 'Intent', 'Flow Type', 
    'Flow Type Simplified', 'Flow Class', 'Sector Code', 'Sector Name', 'Funding Agencies', 
    'Funding Agencies Type', 'Direct Receiving Agencies', 'Direct Receiving Agencies Type', 
    'Direct Receiving Agencies Country of Inc', 'Implementing Agencies Type', 'Commitment Date Estimated', 
    'Interest at T0', 'Management Fee', 'Loan Template Category', 'Commitment Fee', 
    'First Loan Repayment Date', 'Last Loan Repayment Date', 'Grant Element (OECD Cash-Flow)', 
    'Grant Element (IMF)', 'Number of Lenders', 'Credit Line', 'Investment', 'Non-Chinese Financier', 
    'Project JV/SPV Chinese Government Ownership', 'Level of Public Liability', 'Total Source Count', 
    'Official Source Count', 'Source URLs', 'Source Titles', 'Source Publishers', 
    'Source Resource Types', 'ODA Eligible Recipient', 'Suspicious', 'Flagged', 'Published', 
    'Last Date Edited (MM/DD/YYYY)', 'Recipient ISO Alpha-2 Country Code', 
    'Recipient OECD Country Code', 'Recipient COW Country Code', 'Recipient UN Country Code',
    'Recipient IMF Country Code', 'Description (with Sources)', 'Verified By Official Source', 
    'Unofficial Source Count', 'Official Funding', 'Active', 'Stage', 'Tuff Initiatives', 
    'Parent Source Organization', 'Source Priority'
]

# Drop only columns that exist in the DataFrame
cleaner_internal_docp_ddc = cleaning_internal_docp_ddc.drop(
    columns=[col for col in columns_to_drop if col in cleaning_internal_docp_ddc.columns]
)

# Add PrimaryKey column
cleaner_internal_docp_ddc['PrimaryKey'] = range(1, len(cleaner_internal_docp_ddc) + 1)
cleaner_internal_docp_ddc.to_csv('cleaner_internal_docp_ddc.csv', index=False)

# Now ready for AI workflow